# Hyper Parameters

In [1]:
notebookName = "BaseModel"
nepochs = 10
batch_size = 4
learning_rate = 0.001

SEED = 20180724
class_names = [ 'road_block', 'walkway', 'road_divider', 'traffic_light']

In [2]:
LIDAR_PC_SHAPE = [4, 35000] # [x,y,z,intensity] x num_of_points
MAP_OBJECT_SHAPE = [1, 30, 2] # num_of_objects x num_of_points x [x, y]

# Base Setting

In [ ]:
import os
from pathlib import Path

from nuscenes.nuscenes import NuScenes
from utils.custom_lidar_api import CustomLidarApi
from utils.custom_map_api_expansion import CustomNuScenesMap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import tqdm

import torch
import torch.nn as nn
from torch.nn import Module
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset, DataLoader

import warnings
warnings.filterwarnings(action='ignore')

%matplotlib inline 

In [ ]:
locations = ['singapore-onenorth', 'singapore-hollandvillage', 'singapore-queenstown', 'boston-seaport']
version = 'v1.0-trainval'
dataroot = 'E:/datasets/nuscenes'

In [ ]:
bmtorch.manual_seed(SEED)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:

PATH = Path(f"./models/{notebookName}")
if os.path.isdir(PATH):
    dir_list = os.listdir(PATH)
    num_files = 0
    while True:
        if os.path.isfile(str(PATH / f"{num_files}")):
            print(num_files)
            num_files += 1
        else:
            break
else:
    os.mkdir(PATH)
    num_files = 0
num_files

In [ ]:
class_dict = dict()

for i, name in enumerate(class_names):
    class_dict[name] = i
class_array = np.eye(len(class_names))

In [ ]:
nusc = NuScenes(version=version, dataroot=dataroot, verbose=True)
ldr_api = CustomLidarApi(nusc)

map_api = dict([])
for location in locations:
    map_api[location] = CustomNuScenesMap(dataroot = dataroot, map_name= location)

In [ ]:
# get all sample token
sample_tokens = []
for scene in nusc.scene:
    token = scene['first_sample_token']
    while token != scene['last_sample_token']:
        sample_tokens.append(token)
        sample = nusc.get('sample', token)
        token = sample['next']
print(len(sample_tokens))

In [ ]:
def train_val_split(sample_tokens, ratio = 0.1, shuffle = True):
    index = np.array(range(len(sample_tokens)))
    index = np.random.choice(index.shape[0], index.shape[0], replace = False)
    
    valid_num = int(index.shape[0] * ratio)
    
    valid_tokens = sample_tokens[:valid_num]
    train_tokens = sample_tokens[valid_num:]
    
    return train_tokens, valid_tokens

train_tokens, valid_tokens = train_val_split(sample_tokens)

print(len(sample_tokens))
print(len(train_tokens))
print(len(valid_tokens))

In [ ]:
class NusceneDataset(Dataset):
    def __init__(self, tokens, nusc, map_api, ldr_api, train = True):
        self.tokens = tokens
        self.nusc = nusc
        self.map_api = map_api
        self.ldr_api = ldr_api
        self.train = train
        
        self.length = len(tokens)
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        token = self.tokens[idx]
        
        sample = self.nusc.get('sample', token)
        scene = self.nusc.get('scene', sample['scene_token'])
        log_meta = self.nusc.get('log', scene['log_token'])
        
        location = log_meta['location']
        sample_data = self.nusc.get('sample_data', sample['data']['LIDAR_TOP'])
        
        pc = self.ldr_api.get_lidar_from_keyframe(token, car_coord = True)
        ego = self.ldr_api.get_egopose_from_keyframe(token)
        cs = self.nusc.get('calibrated_sensor', sample_data['calibrated_sensor_token'])
        structures = self.map_api[log_meta['location']].get_closest_structures(class_names, ego, global_coord=False)
        
        X = pc.points
        if self.train:
            classes, objects = self.get_label(structures)
        else:
            classes, objects = self.get_label(list())
        
        return X, classes, objects
    
    def get_label(self, structures):
        classes = torch.zeros((64, 4))
        objects = torch.zeros((64,30,2))
        
        for i, struct in enumerate(structures):
            clas = struct['class']
            nodes = struct['nodes']
            
            classes[i,:] = torch.Tensor(class_array[class_dict[clas], :])
            objects[i,:,:] = torch.Tensor(nodes.reshape(1, 30, 2))
        
        return classes, objects
        

train_dataset = NusceneDataset(tokens = train_tokens, nusc = nusc, map_api = map_api, ldr_api = ldr_api)
valid_dataset = NusceneDataset(tokens = valid_tokens, nusc = nusc, map_api = map_api, ldr_api = ldr_api)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)

In [ ]:
class BaseModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pass
    def forward(self, x):
        return x

In [ ]:
class BaseLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pass
    def forward(self, x):
        return x

In [ ]:
model = BaseModel()
model.to(device)

loss_func= BaseLoss()
optimizer = optim.SGD(model.parameters())

In [ ]:
def train(epoch, progress_log):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    mean_loss = 0
    data_num = 0
    
    for X, c, y in progress_log:
        
        X = X.to(device)
        c = c.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        c_hat, y_hat = model(X)  # 데이터를 입력하고 출력을 계산
        loss = loss_func(c_hat, y_hat, c, y)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        mean_loss += loss
        data_num += X.shape[0]
        
    mean_loss /= data_num
    
    return mean_loss

In [ ]:
def valid(epoch, progress_log):
    model.eval()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    mean_loss = 0
    data_num = 0
    
    with torch.no_grad():
        for X, c, y in progress_log:

            X = X.to(device)
            c = c.to(device)
            y = y.to(device)

            c_hat, y_hat = model(X)  # 데이터를 입력하고 출력을 계산
            loss = loss_func(c_hat, y_hat, c, y)  # 출력과 훈련 데이터 정답 간의 오차를 계산

            mean_loss += loss
            data_num += X.shape[0]
        
    mean_loss /= data_num
    
    return mean_loss

In [ ]:
def test(epoch, progress_log):
    model.eval()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    C_hat = []
    Y_hat = []
    
    with torch.no_grad():
        for X, _, _ in progress_log:

            X = X.to(device)

            c_hat, y_hat = model(X)  # 데이터를 입력하고 출력을 계산
            C_hat.append(c_hat)
            Y_hat.append(y_hat)
        
    C_hat = np.concatenate(C_hat)
    Y_hat = np.concatenate(Y_hat)
    
    return C_hat, Y_hat

In [ ]:
train_loss_list = []
valid_loss_list = []

patience_count = 0
min_valid_loss = np.inf
checkpoint_name = ""

if not os.path.isdir(f"./models/{notebookName}/model-{num_files}_checkpoint/"):
    os.mkdir(f"./models/{notebookName}/model-{num_files}_checkpoint/")
    
prog_epoch = tqdm(range(0, nepochs), position = 0, desc = 'EPOCH')
for epoch in prog_epoch:
    print( "-------------------------------------------------------")
    print(f"|EPOCH: {epoch+1}/{nepochs}")
    prog_train = tqdm(train_loader, desc = 'TRAIN', leave = False)
    prog_valid = tqdm(valid_loader, desc = 'VALID', leave = False)

    train_loss = train(epoch, prog_train)
    valid_loss = valid(prog_valid)
    
    if valid_gpsscore < min_valid_score:
        print(f"|{epoch+1}-th model is checked!, *model-{epoch}-{valid_gpsscore}.pth*")
        min_valid_score= valid_gpsscore
        checkpoint_name = f"./models/{notebookName}/model-{num_files}_checkpoint/model-{epoch}-{valid_gpsscore}.pth"
        torch.save(model.state_dict(), checkpoint_name)
    else:
        patience_count+=1
        if(patience_count > max_patience_count):
            break
    
    train_loss_list.append(train_loss)
    valid_loss_list.append(valid_loss)
    
    print(f"|TRAIN: loss={train_loss:.6f}|")
    print(f"|VALID: loss={valid_loss:.6f}|")


history = dict()
history['train_loss'] = train_loss_list
history['valid_loss'] = valid_loss_list